In [1]:
import numpy as np

#get energy function for 2D ising
def Energy(latt): #fast energy calc w no for loops thanks to chat gtp
    Ene = -np.sum(latt*(np.roll(latt,1,axis=0) + np.roll(latt,-1,axis=0) + np.roll(latt,1,axis=1) + np.roll(latt,-1,axis=1)))
    return int(Ene/2.)

#figure out energy spectrum
def calc_E_spectrum(n):
    n_squared = n**2
    #create flat lattice and square lattice for minimum energy
    min_lat_flat = np.array([1]*n_squared)
    min_lat = min_lat_flat.reshape((n,n))
    Emin = float(Energy(min_lat))
    
    #Max energy is every other spin is flipped
    min_lat_flat[::2] = -1
    max_lat = min_lat_flat.reshape((n,n))
    if n%2 == 0:
        max_lat[::2] = max_lat[::2]*-1
    Emax = float(Energy(max_lat))

    #flip a spin to see how it is discritized
    flip_lat = np.array([-1] + [1]*(n_squared - 1)).reshape((n,n))
    Eflip = Energy(flip_lat)
    
    #calculate the change in energy
    dE = (Eflip-Emin)/2
    
    #generate the spectrum
    print(f'Emax = {Emax}, Emin = {Emin}, dE = {dE}')
    E_spectrum = np.arange(Emin,Emax+dE,dE)
    return np.delete(E_spectrum, [1,-2]) #second and second to last energies are not reachable so they are removed

def Fast_WL(N):
    N_squared = N**2
    lattice = (2*np.random.randint(2,size=(1,N_squared))[0]-1).reshape((N,N))

    E_spectrum = calc_E_spectrum(N)
    #E_spectrum = E_spectrum[E_spectrum >= 0] #take only half the spectrum its symmetric!
    
    #following line is to test overlap!
    E_spectrum = E_spectrum[E_spectrum >= 0.1*E_spectrum[0]] #take only half the spectrum its symmetric!

    print(E_spectrum)
    
    E_curr = Energy(lattice)
    while E_curr <= 0:
        lattice = (2*np.random.randint(2,size=(1,N_squared))[0]-1).reshape((N,N))
        E_curr = Energy(lattice)

    #density of states has form of energy spectrum!
    log_g = np.zeros_like(E_spectrum) #would be all 0, starting g at 1

    #initialize histogram -> what will be judged if flat
    histogram = np.zeros_like(E_spectrum)

    #modification factor
    f = 1 #ln(e) = 1
    tot = 0
    c=0
    while f>10**-8:
        #chooose random energy
        prop_bits = (np.random.randint(0,N), np.random.randint(0,N))
        site = lattice[prop_bits] # its spin
        i, j = prop_bits
        nearest_neighbors = lattice[(i+1)%N, j] + lattice[i,(j+1)%N] + lattice[(i-1)%N,j] + lattice[i,(j-1)%N]
        E_proposed = E_curr + 2*site*nearest_neighbors       # The energy of the tryed step
        
        if int(E_proposed) == int(E_spectrum[-2]+2): #makes sure it doesn't transition to 196 but to 200
            E_proposed = E_spectrum[-1]

        if E_proposed < E_spectrum[0]:
            continue
            
        #where is the energy value for the current and proposed
        ind_E_curr = np.where(E_spectrum==E_curr)[0][0]
        ind_E_proposed = np.where(E_spectrum==E_proposed)[0][0]

        #calculate the density of states
        g_curr = log_g[ind_E_curr]
        g_proposed = log_g[ind_E_proposed]

        #calc prob of acceptance
        prob_acceptance = np.exp(g_curr - g_proposed) #g_curr/g_proposed
        r = np.random.uniform(0,1)
        if r < np.minimum(1,prob_acceptance):
            E_curr = E_proposed
            ind_E_curr = ind_E_proposed
            lattice[prop_bits] = lattice[prop_bits]*-1
            
        #else:
            #E_proposed = E_curr
            
        histogram[ind_E_curr] += 1
        log_g[ind_E_curr] += f #add f because working w natural log, log(g*f) = log(g)+log(f)
        tot+=1
        if np.min(histogram) >= np.mean(histogram)*0.8:
            c+=1
            print(c,'success', tot)
            #plt.bar(E_spectrum,histogram,width=4)
            histogram = np.zeros_like(E_spectrum)
            f *= 0.5
    print(f'Done with {N}')
    log_g = log_g[abs(int(E_spectrum[0]/4)):] #burn off the overlap
    return np.concatenate((log_g[1:][::-1],log_g), axis=0)



In [ ]:
log_g = Fast_WL(32)